In [ ]:
import scanpy as sc
import anndata as an
import os
import glob

# Data load

In [ ]:
## Load CellRanger output files into AnnData objects
base_path = r"G:\My Drive\projects\cflows\cellranger_out\cellranger_out"

# Create dictionary to hold all AnnData objects
adata_dict = {}

# Load h5 files from the the root
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith(".h5"):
            full_path = os.path.join(root, file)
            sample_name = os.path.basename(root)  # Use folder name as sample ID
            print(f"Loading: {sample_name} from {full_path}")
            try:
                adata = sc.read_10x_h5(full_path)
                adata.var_names_make_unique()
                adata_dict[sample_name] = adata
            except Exception as e:
                print(f"Failed to load {sample_name}: {e}") 

In [ ]:
# Dimensions of each sample
for sample, adata in adata_dict.items():
    print(f"{sample}: {adata.shape}")

# QC and filtering

In [ ]:
# Perform QC, filtering, and visualization per sample

# Relevant metrics
for sample_name, adata in adata_dict.items():
    print(f"\nProcessing sample: {sample_name}")
    print(f"Original shape: {adata.shape}")

    # Annotate mitochondrial genes
    adata.var['mt'] = adata.var_names.str.startswith('MT-')
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], inplace=True)

    #Make a copy of the unfiltered data for visualization
    adata_raw = adata.copy()

    # Violin plot of unfiltered data
    sc.pl.violin(adata_raw, 
                 ['total_counts', 'n_genes_by_counts', 'pct_counts_mt'],
                 jitter = 0.4, 
                 multi_panel=True)

    # Filter genes only exprssed in less than 3 cells
    sc.pp.filter_genes(adata, min_cells=3)

    # Filter cells based on gene count and mitochondrial content
    # cells containing more than 200 unique genes
    # cells containing less than 5% of mitochondrial genes
    adata = adata[adata.obs.n_genes_by_counts > 200, :]
    adata = adata[adata.obs.pct_counts_mt < 10, :]

    adata_dict[sample_name] = adata
    print(f"Filtered shape: {adata.shape}")

    #QC metrics after filtering
    sc.pl.violin(adata, 
                 ['total_counts', 'n_genes_by_counts', 'pct_counts_mt'],
                 jitter =0.4, 
                 multi_panel=True)

    sc.pl.scatter(adata, x='total_counts', y ='pct_counts_mt')
    sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color='pct_counts_mt')

print("Done! All samples processed")

In [ ]:
# Doublets identification
# Scrublet gives each cell a doublet score, by simulating doublets combining different cells expressions

for sample_name, adata in adata_dict.items():
    print(f"Running Scrublet on {sample_name}...")
    sc.pp.scrublet(adata) 
    print(adata.obs[['doublet_score', 'predicted_doublet']].head())


# Normalization
Count depth scaling and log1p transformation

In [ ]:
#Normalizing to median total counts
for sample_name, adata in adata_dict.items():
    print(f"Normalizing {sample_name}...")
    sc.pp.normalize_total(adata)
    sc.pp.log1p(adata)
    print("All data has been normalized")

# Dimensionality reduction

In [ ]:
#Find first features with the highest variability, stored in .var

for sample_name, adata in adata_dict.items():
    print(f"Searching high variable genes for {sample_name}..")
    sc.pp.highly_variable_genes(adata, n_top_genes = 2000)
    sc.pl.highly_variable_genes(adata)



In [ ]:
# PCA analysis and elbow plot
# PCA coordinates and their variance contribution stored .uns
for sample_name, adata in adata_dict.items():
    print(f"Performing PCA for {sample_name}.. ")
    sc.tl.pca(adata)
    sc.pl.pca_variance_ratio(adata, n_pcs = 50, log =True)
    sc.pl.pca(
    adata,
    color="pct_counts_mt",
)


In [ ]:

# Nearest neighbours, UMAP and visualization
for sample_name, adata in adata_dict.items():
    print(f"UMAP for {sample_name}:")
    sc.pp.neighbors(adata, n_pcs=10) # adjust PCs 
    sc.tl.umap(adata)
    sc.pl.umap(adata,
    color = "pct_counts_mt",
    size = 10,
)


In [ ]:
# The numnber of iterations will affect Leiden cluster queality and modularity
# you can adjust the resolution parameter inside leiden function
for sample_name, adata in adata_dict.items():
    print(f"Running Leiden on {sample_name}...")
    sc.tl.leiden(adata, flavor="igraph", n_iterations=2)
    sc.pl.umap(adata, color=["leiden"], title=f"{sample_name} - Leiden")


# PHATE visualization

In [ ]:
# For trajectory analysis with PHATE we will concatenate again the samples
from anndata import AnnData
adata_combined = an.concat(
    adata_dict,
    label = "sample",           
    index_unique = None         
)

adata_combined.obs

In [ ]:
import phate
import scprep # phate needs scprep

phate_op = phate.PHATE(n_components=10)
phate_embedding = phate_op.fit_transform(data_for_phate)

#Store PHATE results in adata
adata.obsm["X_phate"] = phate_embedding
sc.pl.embedding(adata, basis="phate", color=["sample", "timepoint", "leiden"])
